In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
from dateutil.relativedelta import relativedelta

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
def merge_datasets(ds1, ds2):
    
    """ Merges two datasets based on MRN and Discharge Date. 
    Integrates in order ds1, ds2.
    For the same column, ds2 will only fill in ds1 blanks.
    """

    merged_df = pd.merge(ds1, ds2, on=['MRN', 'Discharge_Date'], how='outer', suffixes=("_ds1", "_ds2"))
    merged_df["Admission_Date_ds1"] = pd.to_datetime(merged_df["Admission_Date_ds1"]).dt.date
    merged_df["Admission_Date_ds2"] = pd.to_datetime(merged_df["Admission_Date_ds2"]).dt.date

    common_columns = set(ds1.columns).intersection(ds2.columns)
    cols_to_ignore = ['MRN', 'Discharge_Date', 'Admission_Date']
    cols_to_fill = common_columns - set(cols_to_ignore)

    for col in cols_to_fill:
        merged_df[col + "_ds1"].fillna(merged_df[col + "_ds2"], inplace=True)
        
    merged_df['Admission_Date_1'] = merged_df.apply(lambda row:                                                         #### Catch missing and inconsistent admission dates
                                              row['Admission_Date_ds1'] if pd.isnull(row['Admission_Date_ds2']) else 
                                              (row['Admission_Date_ds2'] if pd.isnull(row['Admission_Date_ds1']) else 
                                               min(row['Admission_Date_ds1'], row['Admission_Date_ds2'])), axis=1)      #### Use the earliest admission date
    
    merged_df.drop(columns=['Admission_Date_ds1', 'Admission_Date_ds2'], inplace=True)
    merged_df.drop(columns=[col + '_ds2' for col in cols_to_fill], inplace=True)
    merged_df.columns = [col.replace('_ds1', '') for col in merged_df.columns]
    merged_df.sort_values(by=['MRN', 'Discharge_Date'], inplace=True)
    merged_df.rename(columns={'Admission_Date_1': 'Admission_Date'}, inplace=True)
    merged_df.reset_index(drop=True, inplace=True)
    
    return merged_df
        

In [ ]:
def merge_suppdata(ds, supp, name): 
    ''' Merging supplementary data: Weight, NT-proBNP, HbA1c
    Keeps all entries within admission and discharge date and has 1 date and 1 value column per entry.
    '''
    
    ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]   
    supp = supp.loc[:, ~supp.columns.str.contains('^Unnamed')]
    
    supp_2 = supp[supp['MRN'].isin(ds['MRN'])]   # Don't need to consider patients that are not in our dataset
    
    merged_addis = pd.merge(ds, supp_2, on=['MRN'], how="left")
    merged_addis = merged_addis.loc[:, ~merged_addis.columns.str.contains('^Unnamed')]
    merged_addis.sort_values(by=['MRN', 'Discharge_Date'], inplace=True)
    
    # Admission and Discahrge test results, accept within 24 hours of admission/discharge rather than just the day
    merged_addis[f'{name}_Admission'] = merged_addis.apply(lambda row: row['Test Result'] if (not pd.isnull(row['RequestDate']) and (not pd.isnull(row['Admission_Date'])) and
                                                                                             (((row['Admission_Date'] + pd.Timedelta(days=1)) == row['RequestDate']) or (row['Admission_Date'] == row['RequestDate']))) else np.nan, axis=1)

    merged_addis[f'{name}_Discharge'] = merged_addis.apply(lambda row: row['Test Result'] if (not pd.isnull(row['RequestDate']) and (not pd.isnull(row['Discharge_Date'])) and
                                                                                             (((row['Discharge_Date'] - pd.Timedelta(days=1)) == row['RequestDate']) or (row['Discharge_Date'] == row['RequestDate']))) else np.nan, axis=1)

    # Just need the admission and discharge data
    cols_to_drop = supp.columns.tolist()  
    cols_to_drop.remove('MRN') 

    merged_addis.drop(columns=cols_to_drop, inplace=True)
                   
    addis_sup = pd.merge(merged_addis, supp_2, on=['MRN'], how="left", suffixes=("_ds", "_supp"))
    
    addis_sup.loc[:, 'RequestDate'] = pd.to_datetime(addis_sup['RequestDate'], format='%Y-%m-%d')
    addis_sup.loc[:, 'Admission_Date'] = pd.to_datetime(addis_sup['Admission_Date'])
    addis_sup.loc[:, 'Discharge_Date'] = pd.to_datetime(addis_sup['Discharge_Date'])
    merged_addis.loc[:, 'Admission_Date'] = pd.to_datetime(merged_addis['Admission_Date'])
    
    # Only need the test results that are within the episode
    filtered_df = addis_sup[
    (addis_sup["RequestDate"] >= addis_sup["Admission_Date"]) &
    (addis_sup["RequestDate"] <= addis_sup['Discharge_Date'])
    ]

    grouped = filtered_df.groupby(["MRN", "Discharge_Date"])


    # Get list of all test results and dates for MRNs within datasets for same discharge date - less inconsistent than admission
    result_df = grouped.agg({
        'RequestDate': list,
        'Test Result': list
    }).reset_index()

    
    result_df.loc[:, 'Discharge_Date'] = pd.to_datetime(result_df['Discharge_Date'])
    

    # Forward and back fill so can then drop duplicates - captures inconsistent dates and deals with multiple episodes
    merged_addis[f'{name}_Admission'] = merged_addis.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].bfill()
    merged_addis[f'{name}_Admission'] = merged_addis.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].ffill()

    merged_addis[f'{name}_Discharge'] = merged_addis.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].bfill()
    merged_addis[f'{name}_Discharge'] = merged_addis.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].ffill()
    merged_addis.drop_duplicates(inplace=True)
    merged_addis.loc[:, 'Discharge_Date'] = pd.to_datetime(merged_addis['Discharge_Date'])
    
    merged_suppdf = pd.merge(merged_addis, result_df, on=["MRN", "Discharge_Date"], how="outer")


    float_rows_mask = (  
        merged_suppdf['Test Result'].isna()    
    )
    
    merged_suppdf_noNATs = merged_suppdf[~float_rows_mask]

    # All test result entries with their dates
    merged_suppdf_noNATs['Combined'] = merged_suppdf_noNATs.apply(lambda row: list(zip(row['RequestDate'], row['Test Result'])), axis=1)

    # Remove multiple entries - a col containing all results per episode
    merged_suppdf_noNATs['UniqueCombined'] = merged_suppdf_noNATs['Combined'].apply(lambda x: list(set(x)))
    merged_suppdf_noNATs['UniqueCombined'] = merged_suppdf_noNATs['UniqueCombined'].apply(lambda x: [list(i) for i in set(map(tuple, x))])
    merged_suppdf_noNATs['UniqueCombined'] = merged_suppdf_noNATs['UniqueCombined'].apply(lambda x: sorted(x, key=lambda pair: pair[0]))

                   
    num_pairs = max(len(pair_list) for pair_list in merged_suppdf_noNATs['UniqueCombined'])

    # Have one column per result and per test date
    for i in range(1, num_pairs + 1):
        merged_suppdf_noNATs[f'{name}_DT {i}'] = merged_suppdf_noNATs['UniqueCombined'].apply(lambda x: x[i-1][0] if len(x) >= i else None)
        merged_suppdf_noNATs[f'{name}_Result {i}'] = merged_suppdf_noNATs['UniqueCombined'].apply(lambda x: x[i-1][1] if len(x) >= i else None)

    merged_suppdf_noNATs.drop(columns=['RequestDate', 'Test Result', 'Combined',], inplace=True)
    merged_suppdf_noNATs.drop_duplicates(inplace=True)

    # Check admission and discharge results are correst
    merged_suppdf_result1 = merged_suppdf_noNATs[(merged_suppdf_noNATs[f'{name}_Admission'].isnull()) | (merged_suppdf_noNATs[f'{name}_Result 1'] == merged_suppdf_noNATs[f'{name}_Admission'])]

    filtered_rows = []
    for index, row in merged_suppdf_result1.iterrows():
        last_non_null = row.dropna().iloc[-1]  # Find the last non-null value in the row, check if this is our value upon discharge - if not, no test was done upon discharge
        if pd.isna(row[f'{name}_Discharge']) or row[f'{name}_Discharge'] == last_non_null:
            filtered_rows.append(index)

    merged_suppdf_checked = merged_suppdf_result1.loc[filtered_rows]

    merged_suppdf_checked[f'{name}_Admission'] = merged_suppdf_checked.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].bfill()
    merged_suppdf_checked[f'{name}_Admission'] = merged_suppdf_checked.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].ffill()

    merged_suppdf_checked[f'{name}_Discharge'] = merged_suppdf_checked.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].bfill()
    merged_suppdf_checked[f'{name}_Discharge'] = merged_suppdf_checked.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].ffill()
    suppdf_checkadmit = merged_suppdf_checked[(merged_suppdf_checked[f'{name}_Admission'].isnull()) | (merged_suppdf_checked[f'{name}_Result 1'] == merged_suppdf_checked[f'{name}_Admission'])]
    suppdf_checkadmit.drop_duplicates(inplace=True)
    finalsupp = pd.merge(suppdf_checkadmit, merged_addis, how='outer')
    finalsupp.drop_duplicates(inplace=True)

    ds.loc[:, 'Discharge_Date'] = pd.to_datetime(ds['Discharge_Date'])

    # Merge all the suplementary data back with the origional dataset now that it is in the format that we want
    finalsupp.loc[:, 'Discharge_Date'] = pd.to_datetime(finalsupp['Discharge_Date'])
        
    finalsupp[f'{name}_Admission'] = finalsupp.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].bfill()
    finalsupp[f'{name}_Admission'] = finalsupp.groupby(['MRN', 'Discharge_Date'])[f'{name}_Admission'].ffill()

    finalsupp[f'{name}_Discharge'] = finalsupp.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].bfill()
    finalsupp[f'{name}_Discharge'] = finalsupp.groupby(['MRN', 'Discharge_Date'])[f'{name}_Discharge'].ffill()
    
    ds_supp = pd.merge(ds, finalsupp, on=['MRN', 'Discharge_Date'], how='right')
    ds_supp.sort_values(by=['MRN', 'Discharge_Date'], inplace=True)

    ds_supp = ds_supp.loc[:, ~ds_supp.columns.str.contains('^Unnamed')]  
    
    # Fill the common columns - will have to ensure same naming first
    common_columns = set(ds.columns).intersection(finalsupp.columns)
    cols_to_ignore = ['MRN', 'Discharge_Date', 'Admission_Date']
    cols_to_fill = common_columns - set(cols_to_ignore)

    for col in cols_to_fill:
        ds_supp[col + "_x"].fillna(ds_supp[col + "_y"], inplace=True)
    
    ds_supp.drop(columns=[col + '_y' for col in cols_to_fill], inplace=True)
    ds_supp.columns = [col.replace('_x', '') for col in ds_supp.columns]

    ds_supp.drop_duplicates(inplace=True)

    # Put date col in terms of days from admission rather than absolute  date
    date_columns = [col for col in ds_supp.columns if col.startswith(f'{name}_DT')]
    ds_supp[date_columns] = ds_supp[date_columns].apply(pd.to_datetime)
    ds_supp['Admission_Date'] = ds_supp['Admission_Date'].apply(pd.to_datetime)
    for col in date_columns:
        ds_supp[col] = (ds_supp[col] - ds_supp['Admission_Date']).dt.days
    
    return ds_supp

In [ ]:
# CDS, Healtheintent, NICOR datasets
CDS_raw = pd.read_csv("../01_Raw/HF_CDS.csv", low_memory=False)
Healtheintent_raw = pd.read_csv("../01_Raw/HF_Healtheintent.csv", low_memory=False)
NICOR_raw = pd.read_csv("../01_Raw/NICOR_NHFA_collated.csv", low_memory=False)

In [ ]:
# supplementary info
raw_NT = pd.read_excel("../01_Raw/NTProBNP Pathology Nov19 to Date.xlsx")
weight = pd.read_csv("../01_Raw/weight_clean.csv")
hba1c = pd.read_csv("../01_Raw/hba1c_clean.csv")

In [ ]:
#Clean NT-pro and prepare for merge
NT_clean = raw_NT.dropna(subset=['PatientNumber', 'Test Result'], how='any')
NT_clean = NT_clean.drop(['OBRExamCodeText', 'OBRExamCodeText.1', 'ObservationDate', 'ReportedDate', 'RequestingClinician', 'EncounterLocationType', 'ResultRange', 'ResultUnits', 'SEX'], axis='columns')
NT_clean = NT_clean[NT_clean['Test Result'].apply(lambda x: bool(re.match(r'^([<>]\d+|[\d]+)$', str(x))))]
NT_clean['RequestDate'] = pd.to_datetime(NT_clean['RequestDate'], format='%Y-%m-%d %H:%M:%S').dt.date
NT_clean = NT_clean.rename(columns={'PatientNumber': 'MRN'})
NT_clean.sort_values(by=['MRN', 'RequestDate'], ascending=False)
NT_clean['MRN'] = NT_clean['MRN'].astype(int)

In [ ]:
#Clean routine data and prepare for merge -- need consistent column names
# CDS
CDS_raw["Admission_Date"] = pd.to_datetime(CDS_raw["Admission Date"], format='%d/%m/%Y %H:%M').dt.date
CDS_raw["Discharge_Date"] = pd.to_datetime(CDS_raw["Discharge Date"], format='%d/%m/%Y %H:%M').dt.date
CDS_raw.rename(columns={'Died_In_Hospital': 'Died_while_Inpatient',
                    'Died_Within30days_OfDischarge': 'Died_within_30days_of_Discharge',
                    'Rockwood_WithinSpell_Score': 'Frailty_Score',
                       'AdmissionMethod': 'Admission_Method'}, inplace=True)
CDS_raw.dropna(subset=['MRN'], inplace=True)
CDS_raw['MRN'] = CDS_raw['MRN'].astype(str)

# Healtheintent
Healtheintent_raw['Admission_Date'] = pd.to_datetime(Healtheintent_raw['Bm_Registration_Date'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
Healtheintent_raw['Discharge_Date'] = pd.to_datetime(Healtheintent_raw['Oa_Discharge_Date'], format='%Y-%m-%d %H:%M:%S.%f').dt.date
Healtheintent_raw['DoB'] = pd.to_datetime(Healtheintent_raw['DoB'], format='%Y-%m-%d').dt.date
Healtheintent_raw['Admission_Date'] = pd.to_datetime(Healtheintent_raw['Admission_Date'])
Healtheintent_raw['DoB'] = pd.to_datetime(Healtheintent_raw['DoB'])
Healtheintent_raw = Healtheintent_raw.dropna(subset=['DoB'])
Healtheintent_raw['Age_on_Admission_calc'] = (Healtheintent_raw['Admission_Date'] - Healtheintent_raw['DoB']) // pd.Timedelta(days=365.25)
Healtheintent_raw.rename(columns={'Aa_MRN': 'MRN',
                                  'Ae_Deceased_Date': 'Deceased_Date',
                                  'Af_Facility': 'Site',
                                  'Ag_Ethnicity': 'Ethnicity',
                                  'Ah_Patient_Age': 'Age_on_Admission',
                                  'Ai_Gender': 'Gender',
                                  'Aj_Died_while_Inpatient': 'Died_while_Inpatient',
                                  'Al_Died_within_30days_of_Discharge': 'Died_within_30days_of_Discharge',
                                  'Ba_Admission_Method': 'Admission_Method',
                                  'Bf_7days_Readmission_': 'Readmission7d',
                                  'Bg_14days_Readmission_': 'Readmission14d',
                                  'Bh_30days_Readmission_': 'Readmission30d',
                                  'Bi_180days_Readmission_': 'Readmission180d',
                                  'Ga_Diagnosis_Confirmed': 'HF_Diagnosis_Confirmed',
                                  'Ia_Frailty_Score_Values': 'Frailty_Score',
                                  'Ja_Palliative_Care_Referral_': 'Palliative_Care_Referral',
                                  'Ka_Cardiac_Rehabilitation_Referral_': 'Cardiac_Rehabilitation_Referral'}, inplace=True)
Healtheintent_raw.dropna(subset=['MRN'], inplace=True)
Healtheintent_raw['MRN'] = Healtheintent_raw['MRN'].astype(str)

# NICOR
NICOR_raw['Admission_Date'] = pd.to_datetime(NICOR_raw['2.00 Date of Admission'], format='%d/%m/%Y').dt.date
NICOR_raw['Discharge_Date'] = pd.to_datetime(NICOR_raw['15.10 Date of discharge or death'], format='%d/%m/%Y').dt.date
NICOR_raw.rename(columns={'1.02 Local Patient Identifier': 'MRN',
                          '1.08 Ethnic Category': 'Ethnicity', 
                          'Age on Admission': 'Age_on_Admission',
                          '1.07 Patient sex': 'Gender',
                          '15.15 Death in hospital': 'Died_while_Inpatient',
                          '14.00 Confirmed diagnosis of heart failure': 'HF_Diagnosis_Confirmed',
                          '15.05 Was a referral to palliative care services made?': 'Palliative_Care_Referral',
                          '15.01 Was a referral to cardiac rehabilitation made?': 'Cardiac_Rehabilitation_Referral',
                          '1.01 Hospital identifier': 'Site',
                          '8.02 Weight (discharge)': 'Weight_Discharge',
                          '8.02a Weight (admission)': 'Weight_Admission'}, inplace=True)
NICOR_raw.dropna(subset=['MRN'], inplace=True)
NICOR_raw['MRN'] = NICOR_raw['MRN'].astype(str)

In [ ]:
# Merge Healtheintent with CDS
merged_df1 = merge_datasets(Healtheintent_raw, CDS_raw)

In [ ]:
# All three databases
merged_df2 = merge_datasets(merged_df1, NICOR_raw)

In [ ]:
# Gender, ethnicity, deceased status doesn't change by episdoe
merged_df2['Ethnicity'] = merged_df2.groupby('MRN')['Ethnicity'].ffill().bfill()
merged_df2['Gender'] = merged_df2.groupby('MRN')['Gender'].ffill().bfill()
merged_df2['Ad_Current_Deceased_Status'] = merged_df2['Deceased_Date'].apply(lambda x: 'Deceased' if pd.notnull(x) else np.nan)
merged_df2['Ad_Current_Deceased_Status'] = merged_df2.groupby('MRN')['Ad_Current_Deceased_Status'].bfill().ffill()
merged_df2['Deceased_Date'] = merged_df2.groupby('MRN')['Deceased_Date'].bfill().ffill()

In [ ]:
# Add NT-pro data to merged datasets
NT_clean['MRN'] = NT_clean['MRN'].astype(str)
df_NTpro = merge_suppdata(merged_df2, NT_clean, 'NTproBNP')

In [ ]:
#### Removing any with negative dates for NTproDT as issues with admission and discharge dates
df_NTpro_corrected = df_NTpro[df_NTpro['NTproBNP_DT 1'] >= 0]

In [ ]:
#df_NTpro_corrected.to_csv("../03_datasets/df_NTpro_corrected.csv")

In [ ]:
# Clean weight data
weight['RequestDate'] = pd.to_datetime(weight['PERFORMED_DT_TM'], format='%d/%m/%Y %H:%M').dt.date
weight.rename(columns={'Aa_MRN': 'MRN',
                      'RESULT_VAL': 'Test Result'}, inplace=True)
weight['MRN'] = weight['MRN'].astype(str)

In [ ]:
# Contains 3 datasets, NTpro, weight
df_weight = merge_suppdata(df_NTpro_corrected, weight, 'weight')

In [ ]:
# Clean HBA1c data
hba1c['RequestDate'] = pd.to_datetime(hba1c['RequestDate'], format='%Y-%m-%d %H:%M:%S').dt.date
hba1c.rename(columns={'PatientNumber': 'MRN'}, inplace=True)
hba1c['MRN'] = hba1c['MRN'].astype(str)

In [ ]:
# Contains 3 datasets, NTpro, weight, hba1c
df_hba1c = merge_suppdata(df_weight, hba1c, 'hba1c')

In [ ]:
df_hba1c.to_csv("../03_datasets/alldata.csv")